### Data downloaded from https://www.kaggle.com/kingburrito666/better-donald-trump-tweets/data

In [11]:
import numpy as np
from collections import Counter

In [14]:
history_length = 3

def process_input(filename):
    text = open(filename).read()
    sequences = [text[i:i+history_length] for i in range(int(len(text)/history_length))]
    stats = Counter(sequences)
    tokens = []
    counts = []
    for i in stats.most_common():
        tokens.append(i[0])
        counts.append(i[1])
    return stats

def next_char(cur, stats):
    seed = cur[1:]
    candidates = []
    candidatec = []
    for k in stats.keys():
        if seed == k[:-1]:
            candidates.append(k)
            candidatec.append(stats[k])
    candidatep = [x/sum(candidatec) for x in candidatec]
    return candidates[np.random.choice(len(candidates), p=candidatep)]

def sample(length, running_state, stats):
    output = ''
    for i in range(length):
        output += running_state[0]
        running_state = next_char(running_state, stats)
    return output

In [15]:
tweets_stats = process_input('tweets.txt')
sample(50, 'oba', tweets_stats)

'obamil sh  dibizonlyised sed new govemit co zvknow'

In [22]:
history_length = 4

tweets_stats = process_input('tweets.txt')
sample(50, 'obam', tweets_stats)

'obamazing scrimes  afters any of the   ver  ters b'

In [23]:
hamlet_stats = process_input('hamlet.txt')
sample(50, 'and ', hamlet_stats)

'and to say we end the mind that fles, and that fle'


* Above method just uses the history upto a certain number of characters to match the characters. But, there is no context of language generation, eventhough we increase the history length and also it is more memorization than text generation. 
> RNN's shine better in such cases.